In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from keras.models import *
import keras.backend as K
from keras.layers import *
from keras.utils import *
from keras.regularizers import *
import numpy as np
import _pickle as pickle

W0503 23:07:24.268933  3688 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [2]:
embed = hub.Module('./embeddings/GUSE')

In [3]:
def GUSE(param):
    return embed(tf.squeeze(tf.cast(param, tf.string)), signature="default", as_dict=True)["default"]

In [4]:
input_layer = Input(shape=(1,), dtype="string")
guse = Lambda(GUSE, output_shape=(512,))(input_layer)
dense1 = Dense(256, activation="relu", kernel_regularizer=l2(0.005))(guse)
dropout1 = Dropout(0.6)(dense1)
dense2 = Dense(128, activation="relu", kernel_regularizer=l2(0.005))(dropout1)
dropout2 = Dropout(0.6)(dense2)
dense3 = Dense(128, activation="relu", kernel_regularizer=l2(0.005))(dropout2)
dropout3 = Dropout(0.6)(dense3)
dense4 = Dense(64, activation="relu", kernel_regularizer=l2(0.005))(dropout3)
dropout4 = Dropout(0.6)(dense4)
output = Dense(2, activation="softmax")(dropout4)
model = Model(inputs=[input_layer], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0503 23:07:26.009303  3688 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
__________

In [5]:
twitter_f = open("twitter_irony_all.pickle", mode="rb")
twitter_data = pickle.load(twitter_f)
a = np.zeros((156,1))
b = np.ones((156,1))
twitter_y = np.concatenate((a,b), axis=1)
twitter_x = np.asarray(twitter_data)
twitter_x = np.reshape(twitter_x, (twitter_x.shape[0],1))

In [6]:
session = tf.Session()
K.set_session(session)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
model.load_weights("guse_model.h5")
scores = model.evaluate(twitter_x, twitter_y)
scores[1]

156/156 [==============================] - 1s 8ms/step


0.8333333333333334

In [37]:
preds = model.predict(twitter_x)
preds

array([[5.88786483e-01, 4.11213577e-01],
       [3.12904477e-01, 6.87095523e-01],
       [9.33587253e-01, 6.64127171e-02],
       [5.67554355e-01, 4.32445675e-01],
       [3.13545376e-01, 6.86454654e-01],
       [3.13274026e-01, 6.86725974e-01],
       [3.13368678e-01, 6.86631322e-01],
       [3.13066751e-01, 6.86933279e-01],
       [5.07478714e-01, 4.92521256e-01],
       [7.70803690e-01, 2.29196280e-01],
       [3.13160241e-01, 6.86839819e-01],
       [3.13133627e-01, 6.86866343e-01],
       [3.13194245e-01, 6.86805725e-01],
       [3.13009471e-01, 6.86990559e-01],
       [3.12792003e-01, 6.87207997e-01],
       [3.26016217e-01, 6.73983812e-01],
       [3.13003510e-01, 6.86996460e-01],
       [3.95813435e-01, 6.04186535e-01],
       [3.13000858e-01, 6.86999083e-01],
       [3.13040197e-01, 6.86959803e-01],
       [3.12775820e-01, 6.87224209e-01],
       [3.13674897e-01, 6.86325133e-01],
       [3.13266307e-01, 6.86733663e-01],
       [3.14398497e-01, 6.85601532e-01],
       [3.133436

In [68]:
twitter_x[7]

array(['My financial state of affairs is depressing and I cannot afford therapy...  or '],
      dtype='<U137')

In [42]:
twitter_x[2]

array(['Love how my Nutrition teacher is fat as shit lmao. She should take some of her own advice '],
      dtype='<U137')

In [71]:
something = ["Nobody can tell what I suffer! But it is always so. Those who do not complain are never pitied.", '']
smt = np.asarray(something)
smt = np.reshape(smt, (smt.shape[0],1))
pred = model.predict(smt)
pred

array([[0.31306905, 0.68693095],
       [0.8369605 , 0.16303955]], dtype=float32)